In [27]:
# visualize in RGB the mixed atoms

import numpy as np
import os
import json


def make_plot(elements, tot_num_atoms, num_effective_atoms, iteration, dict_list):
    # delete O_pos_one.txt and weights_one.txt if they exist
    file_path = 'O_pos_one.txt'
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File '{file_path}' does not exist, so it cannot be deleted.")
    file_path = 'weights_one.txt'
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File '{file_path}' does not exist, so it cannot be deleted.")
    
    # create O_pos_one.txt
    with open('OH_pos.txt', 'r') as file:
        all_lines = file.readlines()
        # Iterate over each line in the document
        for line_number, line in enumerate(all_lines):
            # Check if the line contains exactly "Iteration 1"
            if line.strip() == f"Iteration {iteration}":
                relevant_lines = all_lines[line_number:line_number + 6 + tot_num_atoms]
                break
        else:
            print(f"The line 'Iteration {iteration}' was not found in O_pos.txt.")
    with open('O_pos_one.txt', 'w') as file:
        file.write(''.join(relevant_lines))

    # create weights_one.txt
    with open('min_progress.txt', 'r') as file:
        all_lines = file.readlines()
        # Iterate over each line in the document
        for line_number, line in enumerate(all_lines):
            # Check if the line contains exactly "Iteration 1"
            if f"Iteration {iteration}" in line.strip():
                relevant_lines = all_lines[line_number:line_number + num_effective_atoms + 2]
                break
        else:
            print(f"The line 'Iteration {iteration}' was not found in min_progress.txt.")
    with open('weights_one.txt', 'w') as file:
        file.write(''.join(relevant_lines))
    
    # extract positions and lattice
    with open('O_pos_one.txt', 'r') as file:
        reading_lattice = False
        reading_positions = False
        lattice = []
        mixpositions = []
        otherpositions = []
        atomtype = []
        for line in file:
            line = line.strip()  # Remove leading and trailing whitespace
            if 'Lattice' in line:
                reading_lattice = True
                continue  # Skip the current iteration and move to the next line
            elif 'Positions' in line:
                reading_lattice = False
                reading_positions = True
                continue
            elif reading_lattice:
                lattice.append(line)
                if len(lattice) == 3:  # Assuming there are always 3 lines for lattice
                    reading_lattice = False
            elif reading_positions:
                if len(mixpositions) + len(otherpositions) < tot_num_atoms:
                    positions_split = line.split()
                    if 'mix' in line:
                        mixpositions.append(f'{positions_split[2]} {positions_split[3]} {positions_split[4]}')
                    else:
                        atomtype.append(positions_split[1])
                        otherpositions.append(f'{positions_split[2]} {positions_split[3]} {positions_split[4]}')
                else:
                    break  # Stop reading file if we have read all required positions

    # convert lattice and positions into vectors
    L = np.zeros((3, 3))
    for i in range(3):
        lattice_split = lattice[i].split()
        # note that columns of lattV are the Bravais lattice vectors
        L[i, :] = np.array([float(lattice_split[0]), float(lattice_split[1]), float(lattice_split[2])])
    mixposV = np.zeros((len(mixpositions), 3))

    for i in range(len(mixpositions)):
        pos_split = mixpositions[i].split()
        mixposV[i, :] = np.array([float(pos_split[0]), float(pos_split[1]), float(pos_split[2])])
    mixP = mixposV.T # make columns the fractional vectors

    otherposV = np.zeros((len(otherpositions), 3))
    for i in range(len(otherpositions)):
        pos_split = otherpositions[i].split()
        otherposV[i, :] = np.array([float(pos_split[0]), float(pos_split[1]), float(pos_split[2])])
    otherP = otherposV.T # make columns the fractional vectors

    # get weights of each mixed atom
    with open('weights_one.txt', 'r') as file:
        lines = file.readlines()
        # Initialize a list to hold the extracted numbers
        weights = []
        # Iterate over each line
        for line in lines:
            split_line = line.strip().split()
            # Check if the line contains information on weights by looking for any of the elements
            if any(elem in split_line for elem in elements):
                parts = split_line
                line_weights = []
                for elem in elements:
                    if elem in parts:  # Check if the element is in the line to avoid errors
                        # Extract the weight for each element and append to the line_weights list
                        weight = float(parts[parts.index(elem) - 1])
                        line_weights.append(weight)
                weights.append(line_weights)
        # extract binding energies
        for line in lines:
            if 'binding' in line:
                split_line = line.strip().split()
                HEA_binding = format(float(split_line[4]), '.2f')
                break
        # Convert the list of lists into a NumPy array
        W = np.array(weights)
    # get cartesian positions of atoms (columns are the positions)
    mixC = L @ mixP
    otherC = L @ otherP

    atom_dict_list = []
    k = 1
    for i in range(len(mixpositions)):
        composition = {}
        for el in elements:
            composition[el] = float(W[i, elements.index(el)])
        composition['O'] = 0.0
        composition['H'] = 0.0
        atom_dict_list.append({
                            "id": k,
                            "position": mixC[:, i],
                            "composition": composition
                            })
        k+=1

    for i in range(len(otherpositions)):
        if atomtype[i] == 'O':
            composition = {}
            for el in elements:
                composition[el] = 0.0
            composition['O'] = 1.0
            composition['H'] = 0.0
            atom_dict_list.append({
                                "id": k,
                                "position": otherC[:, i],
                                "composition": composition
                                })
        elif atomtype[i] == 'H':
            composition = {}
            for el in elements:
                composition[el] = 0.0
            composition['O'] = 0.0
            composition['H'] = 1.0
            atom_dict_list.append({
                                "id": k,
                                "position": otherC[:, i],
                                "composition": composition
                                })
            
        k+=1
    
    dict_list.append({
                        "frame": iteration,
                        "binding": float(HEA_binding),
                        "atoms": atom_dict_list
                    })
    
    return dict_list, L



#############################################################
elements = ['Co', 'Cr', 'Ni', 'V']
numinterp = 1

for run_iter in range(1, 19):
    try:
        os.chdir(f'/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/EAT/Paper_EAT_2025/Blender/EAT_noMag/run{run_iter}+1_6')
    except:
        continue

    with open('min_progress.txt', 'r') as file:
        file_lines = file.readlines()
    for line in file_lines:
        if 'Iteration' in line:
            num_iterations = int(line.strip().split()[1][:-1])

    with open('OH_pos.txt', 'r') as file:
        file_lines = file.readlines()
    tot_num_atoms = 0
    num_effective_atoms = 0
    for line in file_lines:
        if line.strip():
            if line.strip().split()[0] == 'ion':
                tot_num_atoms += 1
            if 'mix' in line:
                num_effective_atoms += 1
            if 'Iteration 2' in line:
                break
    
    dict_list = []
    for i in range(1, num_iterations+1):
        dict_list, lattice_vectors = make_plot(elements, tot_num_atoms, num_effective_atoms, i, dict_list)

    elements_full = elements + ['O', 'H']

    # interpolate between frames
    x = np.linspace(0, 1, numinterp+1)
    interp_list = []
    for j in range(num_iterations-1):
        for i in range(numinterp):
            atom_list = []
            for k in range(tot_num_atoms):
                new_atom_dict = {}
                new_atom_dict["id"] = k+1
                atoms_dict_comp = {}
                for element in elements_full:
                    atoms_dict_comp[element] = x[i] * (dict_list[j+1]['atoms'][k]['composition'][element] - dict_list[j]['atoms'][k]['composition'][element]) + dict_list[j]['atoms'][k]['composition'][element]
                new_atom_dict['composition'] = atoms_dict_comp
                position_temp = x[i] * (dict_list[j+1]['atoms'][k]['position'] - dict_list[j]['atoms'][k]['position']) + dict_list[j]['atoms'][k]['position']
                new_atom_dict["position"] = position_temp.tolist()
                atom_list.append(new_atom_dict)

            interp_list.append({
                                "frame": j * numinterp + i,
                                "binding": x[i] * (dict_list[j+1]['binding'] - dict_list[j]['binding']) + dict_list[j]['binding'],
                                "atoms": atom_list
            })

    # Final frame
    atom_list = []
    for k in range(tot_num_atoms):
        new_atom_dict = {}
        new_atom_dict["id"] = k+1
        atoms_dict_comp = {}
        for element in elements_full:
            atoms_dict_comp[element] = dict_list[-1]['atoms'][k]['composition'][element]
        new_atom_dict['composition'] = atoms_dict_comp
        position_temp = dict_list[-1]['atoms'][k]['position']
        new_atom_dict["position"] = position_temp.tolist()
        atom_list.append(new_atom_dict)
    interp_list.append({
                        "frame": num_iterations * numinterp + numinterp-1,
                        "binding": dict_list[-1]['binding'],
                        "atoms": atom_list
            })  # Append the last frame
    
    # # extend unit cell in x-y direction
    # interp_list_extended = []
    # for entry in interp_list:
    #     atom_list = []
    #     for k in range(tot_num_atoms):
    #         for i in range(2):
    #             for j in range(2):
    #                 new_atom_dict = {}
    #                 new_atom_dict["id"] = k*(2*2) + (i+1)*2 + (j+1) - 2
    #                 new_atom_dict['composition'] = entry["atoms"][k]["composition"]
    #                 position_temp = np.array(entry['atoms'][k]['position']) + i * lattice_vectors[0, :] + j * lattice_vectors[1, :]
    #                 new_atom_dict["position"] = position_temp.tolist()
    #                 atom_list.append(new_atom_dict)
    #     interp_list_extended.append({
    #                             "frame": entry['frame'],
    #                             "binding": entry['binding'],
    #                             "atoms": atom_list
    #         })



    element_colors = {
        'Co': (0.0, 1.0, 1.0),  # Cyan
        'Cr': (0.0, 1.0, 0.0),  # Green
        'Ni': (0.0, 0.0, 1.0),  # Blue
        'V': (1.0, 1.0, 0.0),  # Yellow
        'O': (1.0, 0.0, 0.0), # Red
        'H': (1.0, 1.0, 1.0) # White
    }

    element_sizes = {
        'Co': 2.2,
        'Cr': 2.2,
        'Ni': 2.2,
        'V': 2.2,
        'O': 1.0,
        'H': 0.6
    }

    # Corrected way to save the data as pickle files
    with open(f'/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/EAT/Paper_EAT_2025/Blender/EAT_noMag/run{run_iter}+1_6/atom_data.json', 'w') as pkl_file:  # Note 'wb' for binary write
        json.dump(interp_list, pkl_file)

    with open(f'/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/EAT/Paper_EAT_2025/Blender/EAT_noMag/run{run_iter}+1_6/element_colors.json', 'w') as pkl_file:
        json.dump(element_colors, pkl_file)

    with open(f'/Users/justint/Library/CloudStorage/OneDrive-Personal/Desktop/Academic Stuff/Arias Research/EAT/Paper_EAT_2025/Blender/EAT_noMag/run{run_iter}+1_6/element_sizes.json', 'w') as pkl_file:
        json.dump(element_sizes, pkl_file)


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Ensure the directory for saving images exists
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Your data array
values = []
for entry in interp_list:
    values.append(entry['binding'])
values = np.array(values)

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)  # Higher DPI for better quality
ax.axis('off')
fig.patch.set_facecolor('white')

# Initialize the text object
text = ax.text(0.5, 0.5, '', fontsize=24, color='black',
               ha='center', va='center', transform=ax.transAxes)

# Loop over each value and save each frame as a .png file
for i, value in enumerate(values):
    # Set the text for the current frame with escaped braces for LaTeX-style formatting
    text.set_text(r'$\Delta G_{{O^*}} - \Delta G_{{HO^*}} = {:.2f} \, \mathrm{{eV}}$'.format(value))
    
    # Save the current frame as a .png file
    filename = os.path.join(output_dir, f"frame_{i:03d}.png")
    plt.savefig(filename, bbox_inches='tight', facecolor='white', dpi=300)
    print(f"Saved {filename}")

plt.close()


Saved frames/frame_000.png
Saved frames/frame_001.png
Saved frames/frame_002.png
Saved frames/frame_003.png
Saved frames/frame_004.png
Saved frames/frame_005.png
Saved frames/frame_006.png
Saved frames/frame_007.png
Saved frames/frame_008.png
Saved frames/frame_009.png
Saved frames/frame_010.png
Saved frames/frame_011.png
Saved frames/frame_012.png
Saved frames/frame_013.png
Saved frames/frame_014.png
Saved frames/frame_015.png
Saved frames/frame_016.png
Saved frames/frame_017.png
Saved frames/frame_018.png
Saved frames/frame_019.png
Saved frames/frame_020.png
Saved frames/frame_021.png
Saved frames/frame_022.png
Saved frames/frame_023.png
Saved frames/frame_024.png
Saved frames/frame_025.png
Saved frames/frame_026.png
Saved frames/frame_027.png
Saved frames/frame_028.png
Saved frames/frame_029.png
Saved frames/frame_030.png
Saved frames/frame_031.png
Saved frames/frame_032.png
Saved frames/frame_033.png
Saved frames/frame_034.png
Saved frames/frame_035.png
Saved frames/frame_036.png
S

In [9]:
import imageio.v2 as imageio
import os

# Directory where the frames are saved
output_dir = "dG_frames"
# Path to save the video file
video_filename = 'numerical_values_animation.mp4'

# Collect the frames in order
frame_filenames = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.png')])

# Set the frames per second for the video
fps = int(6)  # Adjust as needed

# Create the video
with imageio.get_writer(video_filename, fps=fps) as writer:
    for filename in frame_filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        print(f"Added {filename} to the video.")

print(f"Video saved as {video_filename}")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1919, 1446) to (1920, 1456) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Added dG_frames/frame_000.png to the video.
Added dG_frames/frame_001.png to the video.
Added dG_frames/frame_002.png to the video.
Added dG_frames/frame_003.png to the video.
Added dG_frames/frame_004.png to the video.
Added dG_frames/frame_005.png to the video.
Added dG_frames/frame_006.png to the video.
Added dG_frames/frame_007.png to the video.
Added dG_frames/frame_008.png to the video.
Added dG_frames/frame_009.png to the video.
Added dG_frames/frame_010.png to the video.
Added dG_frames/frame_011.png to the video.
Added dG_frames/frame_012.png to the video.
Added dG_frames/frame_013.png to the video.
Added dG_frames/frame_014.png to the video.
Added dG_frames/frame_015.png to the video.
Added dG_frames/frame_016.png to the video.
Added dG_frames/frame_017.png to the video.
Added dG_frames/frame_018.png to the video.
Added dG_frames/frame_019.png to the video.
Added dG_frames/frame_020.png to the video.
Added dG_frames/frame_021.png to the video.
Added dG_frames/frame_022.png to